In [1]:
import tabula
import pandas as pd 
import datetime

In [39]:
def to_date(x):
    try:
        x = list(map(int, x.split('-')))
        date = datetime.date(x[0],x[1],x[2])
    except Exception as e:
        print(e) 
        date = None

    return date


def extract_pdf(pdf_url):
    df = pd.concat(tabula.read_pdf(pdf_url, pages = 'all'))
    df['date_payment_confirmed'] = df['date_payment_confirmed'].apply(to_date)
    df['expiry_date'] = pd.to_datetime(df['expiry_date'],format = '%m/%y',errors= 'coerce').dt.date
    return df
    

In [40]:
pdf_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'

card_data = extract_pdf(pdf_url)

invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'GTC9KBWJO9'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'DJIXF1AFAZ'
invalid literal for int() with base 10: 'December 2021 17'
invalid literal for int() with base 10: '2005 July 01'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'H2PCQP4W50'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'XTD27ANR5Q'
invalid literal for int() with base 10: '7VGB4DA1WI'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'NULL'
invalid literal for int() with base 10: 'RLQYRRYHPU'
invalid literal for int() with base 10: 'December 2000 01'
invalid literal for int() with base 10: '2008 May 11'
invalid literal for int() with b

In [13]:
card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15309 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 598.0+ KB


In [14]:
card_data

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,2026-09-01,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,2023-10-01,American Express,2001-06-18
2,3529023891650490,2023-06-01,JCB 16 digit,2000-12-26
3,213142929492281,2027-09-01,JCB 15 digit,2011-02-12
4,502067329974,2025-10-01,Maestro,1997-03-13
...,...,...,...,...
14,180036921556789,2028-12-01,JCB 15 digit,1997-06-06
15,180018030448512,2024-11-01,JCB 15 digit,2004-06-16
16,3569953313547220,2024-04-01,JCB 16 digit,2020-02-05
17,4444521712606810,2027-06-01,VISA 16 digit,2008-06-16


In [15]:
card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15309 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15284 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 598.0+ KB


In [16]:
card_data.isna().sum()

card_number                0
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [17]:
card_data['card_number'].astype(int)

ValueError: invalid literal for int() with base 10: '?4971858637664481'

In [25]:
def clean_questionmark(x):
    try:
        if x == "NULL":
            return None
        x = str(x).replace('?',"")
        return x 
    except Exception as e:
        print(e) 



In [28]:
card_data['card_number'] = card_data['card_number'].apply(clean_questionmark)

In [29]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [35]:
(card_data['card_provider'] == 'NULL').sum()

11

In [34]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider              0
date_payment_confirmed    33
dtype: int64

In [36]:
card_data.loc[card_data['card_provider'] == 'NULL', 'card_provider'] = None

In [37]:
card_data.isna().sum()

card_number               11
expiry_date               25
card_provider             11
date_payment_confirmed    33
dtype: int64

In [38]:
def clean_card_data(card_data):
    card_data['card_number'] = card_data['card_number'].apply(clean_questionmark)
    card_data.loc[card_data['card_provider'] == 'NULL', 'card_provider'] = None
    return card_data.dropna()

    
    


In [41]:
new_data = clean_card_data(card_data)

In [42]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15276 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15276 non-null  object
 1   expiry_date             15276 non-null  object
 2   card_provider           15276 non-null  object
 3   date_payment_confirmed  15276 non-null  object
dtypes: object(4)
memory usage: 596.7+ KB


In [43]:
new_data.isna().sum()

card_number               0
expiry_date               0
card_provider             0
date_payment_confirmed    0
dtype: int64

In [45]:
(new_data['card_number']== 'NULL').sum()

0

In [1]:
from data_extraction import DataExtractor

    

In [2]:
extractor = DataExtractor("www.test.com")

In [3]:
extractor.retrieve_stores_data('https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}')

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
2,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
3,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
4,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
447,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
448,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
449,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe


In [57]:
extractor.list_number_of_stores('https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores')


{'statusCode': 200, 'number_stores': 451}

In [58]:
dir(extractor)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'header_dict',
 'list_number_of_stores',
 'pdf_url',
 'read_rds_table',
 'retrieve_pdf_data',
 'to_date']